In [164]:
# loading libraries 
import pandas as pd
import numpy as np

In [165]:
#read and display data
df1 =pd.read_csv("resources/NY-House-Dataset_suburb.csv")
df2 = pd.read_csv('resources/housing_geocode_extraction.csv')
print(display(df1.head(5)))
print(display(df2.head(5)))

,BROKERTITLE,TYPE,PRICE,BEDS,BATH,PROPERTYSQFT,ADDRESS,STATE,MAIN_ADDRESS,ADMINISTRATIVE_AREA_LEVEL_2,LOCALITY,SUBLOCALITY,suburb,LONG_NAME,FORMATTED_ADDRESS,LATITUDE,LONGITUDE
0,Brokered by ANNE LOPA REAL ESTATE,House for sale,2147483647,7,6.000000,10000.000000,6659-6675 Amboy Rd,"New York, NY 10309","6659-6675 Amboy RdNew York, NY 10309",United States,New York,Richmond County,Staten Island,Amboy Road,"6659 Amboy Rd, Staten Island, NY 10309, USA",40.518484,-74.224418
1,Brokered by Serhant,Condo for sale,195000000,7,10.000000,17545.000000,Central Park Tower Penthouse-217 W 57th New Yo...,"New York, NY 10019",Central Park Tower Penthouse-217 W 57th New Yo...,United States,New York,New York County,Manhattan,West 57th Street,"217 W 57th St, New York, NY 10019, USA",40.766393,-73.980991
2,Brokered by Sotheby's International Realty - E...,Townhouse for sale,65000000,3,2.373861,15200.000000,4 E 79th St,"New York, NY 10075","4 E 79th StNew York, NY 10075",United States,New York,New York County,Manhattan,East 79th Street,"4 E 79th St, New York, NY 10075, USA",40.776561,-73.963317
3,Brokered by COMPASS,Co-op for sale,60000000,8,8.000000,2184.207862,960 5th Ave Unit 12,"Manhattan, NY 10075","960 5th Ave Unit 12Manhattan, NY 10075",New York,New York County,New York,Manhattan,960,"960 5th Ave #12, New York, NY 10075, USA",40.775642,-73.964247
4,Brokered by Douglas Elliman - 575 Madison Ave,House for sale,56000000,11,10.000000,24000.000000,9 W 54th St,"New York, NY 10019","9 W 54th StNew York, NY 10019",United States,New York,New York County,Manhattan,West 54th Street,"9 W 54th St, New York, NY 10019, USA",40.761600,-73.976191


None


,LATITUDE,LONGITUDE,STATE,LOCALITY,SUBLOCALITY,POSTCODE,PROPERTY_CATEGORY
0,40.761255,-73.974483,New York,New York,Manhattan,10022,catering.restaurant
1,40.766393,-73.980991,New York,New York,Manhattan,10019,building.residential
2,40.541805,-74.196109,New York,New York,Staten Island,10312,building.residential
3,40.761398,-73.974613,New York,New York,Manhattan,10022,catering.bar
4,40.767224,-73.969856,New York,New York,Manhattan,10065,building


None


In [166]:
# Clean sublocality and state columns in both datasets
df1['SUBLOCALITY'] = df1['SUBLOCALITY'].str.strip().str.lower()
df1['STATE'] = df1['STATE'].str.strip().str.lower()
df2['SUBLOCALITY'] = df2['SUBLOCALITY'].str.strip().str.lower()
df2['STATE'] = df2['STATE'].str.strip().str.lower()



In [167]:
# Drop duplicates in the key column
df1 = df1.drop_duplicates(subset=['LATITUDE', 'LONGITUDE'])
df2 = df2.drop_duplicates(subset=['LATITUDE', 'LONGITUDE'])

In [168]:
# Merge datasets on LATITUDE and LONGITUDE
merged_df = pd.merge(df1, df2, on=['LATITUDE', 'LONGITUDE'], how='inner', suffixes=('_orig', '_ext'))

In [169]:
#Inspect Overlapping Columns

duplicate_cols = [col for col in merged_df.columns if col.endswith('_orig') or col.endswith('_ext')]
print(duplicate_cols)


['STATE_orig', 'LOCALITY_orig', 'SUBLOCALITY_orig', 'STATE_ext', 'LOCALITY_ext', 'SUBLOCALITY_ext']


In [170]:
merged_df = merged_df.drop(columns=['STATE_orig', 'LOCALITY_orig', 'SUBLOCALITY_orig'])

In [171]:
#check thecolumns names
merged_df.columns

Index(['BROKERTITLE', 'TYPE', 'PRICE', 'BEDS', 'BATH', 'PROPERTYSQFT',
       'ADDRESS', 'MAIN_ADDRESS', 'ADMINISTRATIVE_AREA_LEVEL_2', 'suburb',
       'LONG_NAME', 'FORMATTED_ADDRESS', 'LATITUDE', 'LONGITUDE', 'STATE_ext',
       'LOCALITY_ext', 'SUBLOCALITY_ext', 'POSTCODE', 'PROPERTY_CATEGORY'],
      dtype='object')

In [172]:
# Select relevant columns
selected_columns = [
    'PRICE', 'BEDS', 'BATH', 'PROPERTYSQFT','suburb',
    'PROPERTY_CATEGORY','TYPE', 'SUBLOCALITY_ext','POSTCODE','STATE_ext','LATITUDE', 'LONGITUDE'
]
merged_df = merged_df[selected_columns]
merged_df.rename(columns={'suburb': 'borough'}, inplace=True)


In [173]:
#check the data shape
merged_df.shape

(4200, 12)

In [174]:
#check the merged data
merged_df.head()


,PRICE,BEDS,BATH,PROPERTYSQFT,borough,PROPERTY_CATEGORY,TYPE,SUBLOCALITY_ext,POSTCODE,STATE_ext,LATITUDE,LONGITUDE
0,2147483647,7,6.000000,10000.000000,Staten Island,building.commercial,House for sale,staten island,10309,new york,40.518484,-74.224418
1,195000000,7,10.000000,17545.000000,Manhattan,building.residential,Condo for sale,manhattan,10019,new york,40.766393,-73.980991
2,65000000,3,2.373861,15200.000000,Manhattan,building,Townhouse for sale,manhattan,10075,new york,40.776561,-73.963317
3,60000000,8,8.000000,2184.207862,Manhattan,building.residential,Co-op for sale,manhattan,10075,new york,40.775642,-73.964247
4,56000000,11,10.000000,24000.000000,Manhattan,building,House for sale,manhattan,10019,new york,40.761600,-73.976191


In [175]:
# Calculate price per square foot
merged_df['PRICE_PER_SQFT'] = merged_df['PRICE'] / merged_df['PROPERTYSQFT']

# Standardize type column
merged_df['TYPE'] = merged_df['TYPE'].str.lower().str.replace(' for sale', '', regex=False)

# Display cleaned and enhanced DataFrame
merged_df.head()

,PRICE,BEDS,BATH,PROPERTYSQFT,borough,PROPERTY_CATEGORY,TYPE,SUBLOCALITY_ext,POSTCODE,STATE_ext,LATITUDE,LONGITUDE,PRICE_PER_SQFT
0,2147483647,7,6.000000,10000.000000,Staten Island,building.commercial,house,staten island,10309,new york,40.518484,-74.224418,214748.364700
1,195000000,7,10.000000,17545.000000,Manhattan,building.residential,condo,manhattan,10019,new york,40.766393,-73.980991,11114.277572
2,65000000,3,2.373861,15200.000000,Manhattan,building,townhouse,manhattan,10075,new york,40.776561,-73.963317,4276.315789
3,60000000,8,8.000000,2184.207862,Manhattan,building.residential,co-op,manhattan,10075,new york,40.775642,-73.964247,27469.913026
4,56000000,11,10.000000,24000.000000,Manhattan,building,house,manhattan,10019,new york,40.761600,-73.976191,2333.333333


In [176]:
# # Standardize column names
# merged_df.columns = merged_df.columns.str.lower().str.replace(' ', '_')
# #check changes
# merged_df.columns

Check for Missing Values:

In [177]:
# Check for missing values in the selected columns
missing_data = merged_df.isnull().sum()
print("Missing Data by Column:")
missing_data


Missing Data by Column:


PRICE                  0
BEDS                   0
BATH                   0
PROPERTYSQFT           0
borough                0
PROPERTY_CATEGORY    641
TYPE                   0
SUBLOCALITY_ext        0
POSTCODE               0
STATE_ext              0
LATITUDE               0
LONGITUDE              0
PRICE_PER_SQFT         0
dtype: int64

In [178]:
# Save the merged DataFrame to a CSV file
df_copy = merged_df.copy()

# Convert column names to lowercase in the original DataFrame
df_copy.columns = df_copy.columns.str.lower()
df_copy.to_csv('resources/Cleaned_data.csv', index=False)

# Confirm the file has been saved
print("File saved as 'Cleaned_data'")

File saved as 'Cleaned_data'


Handle Missing Values:

In [179]:
merged_df.columns

Index(['PRICE', 'BEDS', 'BATH', 'PROPERTYSQFT', 'borough', 'PROPERTY_CATEGORY',
       'TYPE', 'SUBLOCALITY_ext', 'POSTCODE', 'STATE_ext', 'LATITUDE',
       'LONGITUDE', 'PRICE_PER_SQFT'],
      dtype='object')

In [180]:
#chech data type
merged_df.dtypes

PRICE                  int64
BEDS                   int64
BATH                 float64
PROPERTYSQFT         float64
borough               object
PROPERTY_CATEGORY     object
TYPE                  object
SUBLOCALITY_ext       object
POSTCODE              object
STATE_ext             object
LATITUDE             float64
LONGITUDE            float64
PRICE_PER_SQFT       float64
dtype: object

In [181]:
# Check for whitespac
merged_df.columns = merged_df.columns.str.strip()

In [182]:
#drope the missing values
merged_df = merged_df.dropna(how='any')

In [183]:
# Check for duplicate rows
duplicates = merged_df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

# Remove duplicates
merged_df = merged_df.drop_duplicates()


Number of duplicate rows: 0


In [184]:
# Define a function to remove outliers using the IQR method
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)  # First quartile
    Q3 = df[column].quantile(0.75)  # Third quartile
    IQR = Q3 - Q1  # Interquartile range
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # Return the dataframe after filtering out outliers
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# List of columns to clean
columns_to_clean = ['PRICE', 'BEDS', 'BATH', 'PROPERTYSQFT', 'PRICE_PER_SQFT']

# Clean outliers for the relevant numerical columns using a loop
for col in columns_to_clean:
    merged_df = remove_outliers(merged_df, col)

# Display the cleaned dataset
merged_df.head()


,PRICE,BEDS,BATH,PROPERTYSQFT,borough,PROPERTY_CATEGORY,TYPE,SUBLOCALITY_ext,POSTCODE,STATE_ext,LATITUDE,LONGITUDE,PRICE_PER_SQFT
529,2800000,5,4.0,3400.0,Brooklyn,building,pending,brooklyn,11218,new york,40.643517,-73.966956,823.529412
533,2800000,3,3.0,3341.0,Brooklyn,building.residential,condo,brooklyn,11211,new york,40.709601,-73.968217,838.072433
534,2799000,3,3.0,2350.0,Brooklyn,building,condo,brooklyn,11215,new york,40.667604,-73.993127,1191.063830
535,2798000,5,3.0,3400.0,Brooklyn,building,multi-family home,brooklyn,11218,new york,40.657854,-73.975525,822.941176
548,2750000,3,3.0,2782.0,Brooklyn,building.commercial,condo,brooklyn,11231,new york,40.681105,-74.009814,988.497484


In [185]:
# check data type
merged_df.dtypes

PRICE                  int64
BEDS                   int64
BATH                 float64
PROPERTYSQFT         float64
borough               object
PROPERTY_CATEGORY     object
TYPE                  object
SUBLOCALITY_ext       object
POSTCODE              object
STATE_ext             object
LATITUDE             float64
LONGITUDE            float64
PRICE_PER_SQFT       float64
dtype: object

In [186]:
#If any column names have leading or trailing spaces
merged_df.columns = merged_df.columns.str.strip()
# Correct Inconsistent Value
merged_df['STATE_ext'] = merged_df['STATE_ext'].str.title()
merged_df['SUBLOCALITY_ext'] = merged_df['SUBLOCALITY_ext'].str.title()


In [187]:
merged_df.head()

,PRICE,BEDS,BATH,PROPERTYSQFT,borough,PROPERTY_CATEGORY,TYPE,SUBLOCALITY_ext,POSTCODE,STATE_ext,LATITUDE,LONGITUDE,PRICE_PER_SQFT
529,2800000,5,4.0,3400.0,Brooklyn,building,pending,Brooklyn,11218,New York,40.643517,-73.966956,823.529412
533,2800000,3,3.0,3341.0,Brooklyn,building.residential,condo,Brooklyn,11211,New York,40.709601,-73.968217,838.072433
534,2799000,3,3.0,2350.0,Brooklyn,building,condo,Brooklyn,11215,New York,40.667604,-73.993127,1191.063830
535,2798000,5,3.0,3400.0,Brooklyn,building,multi-family home,Brooklyn,11218,New York,40.657854,-73.975525,822.941176
548,2750000,3,3.0,2782.0,Brooklyn,building.commercial,condo,Brooklyn,11231,New York,40.681105,-74.009814,988.497484


In [188]:
merged_df['PROPERTY_CATEGORY'].unique()

array(['building', 'building.residential', 'building.commercial',
       'commercial.clothing', 'building.garage', 'accommodation.hotel',
       'tourism.sights', 'public_transport.bus', 'leisure.playground',
       'commercial.furniture_and_interior',
       'commercial;service.cleaning.dry_cleaning',
       'commercial.discount_store', 'catering.fast_food', 'catering.cafe',
       'commercial', 'commercial.health_and_beauty;service.beauty.spa',
       'building.healthcare', 'catering.restaurant', 'building.office',
       'service.vehicle.fuel', 'commercial.pet;pet.shop',
       'commercial;service.beauty.hairdresser', 'sport.sports_centre',
       'commercial.convenience', 'commercial;rental.storage',
       'catering.ice_cream', 'catering.bar',
       'commercial.food_and_drink.bakery', 'healthcare.hospital'],
      dtype=object)

In [189]:
# Define mapping to major categories
# Define mapping to major categories
category_mapping = {
    'building.residential': 'Residential',
    'accommodation.hotel': 'Hospitality',
    'healthcare.hospital': 'Other',  # 
    'building.commercial': 'Commercial',
    'commercial': 'Commercial',
    'commercial;service.beauty.hairdresser': 'Commercial',
    'commercial.health_and_beauty;service.beauty.spa': 'Commercial',
    'commercial.furniture_and_interior': 'Commercial',
    'commercial.pet;pet.shop': 'Commercial',
    'commercial.convenience': 'Commercial',
    'catering.restaurant': 'Commercial',
    'catering.bar': 'Commercial',
    'catering.fast_food': 'Commercial',
    'catering.ice_cream': 'Commercial',
    'catering.cafe': 'Commercial',
    'building.office': 'Commercial',
    'building.healthcare': 'Other',  
    'commercial.clothing': 'Commercial',
    'commercial.discount_store': 'Commercial',
    'service.vehicle.fuel': 'Commercial',
    'commercial;rental.storage': 'Commercial',
    'public_transport.bus': 'Other',
    'tourism.sights': 'Other',
    'sport.sports_centre': 'Other',
    'leisure.playground': 'Other',
    'education.university': 'Other',
    'building.garage': 'Other',
    'building': 'Residential',  
    'commercial;service.cleaning.dry_cleaning': 'Commercial',
}
# Function to map categories
def map_category(category):
    return category_mapping.get(category, 'Other')

# Apply the mapping
merged_df['PROPERTY_CATEGORY'] = merged_df['PROPERTY_CATEGORY'].apply(map_category)

In [190]:
merged_df['PROPERTY_CATEGORY'].unique()

array(['Residential', 'Commercial', 'Other', 'Hospitality'], dtype=object)

In [191]:
# #Drop any rows with missing or unclassified values:
merged_df = merged_df.dropna(subset=['TYPE', 'PROPERTY_CATEGORY'])


In [192]:
merged_df.shape

(2710, 13)

In [193]:
merged_df.head()

,PRICE,BEDS,BATH,PROPERTYSQFT,borough,PROPERTY_CATEGORY,TYPE,SUBLOCALITY_ext,POSTCODE,STATE_ext,LATITUDE,LONGITUDE,PRICE_PER_SQFT
529,2800000,5,4.0,3400.0,Brooklyn,Residential,pending,Brooklyn,11218,New York,40.643517,-73.966956,823.529412
533,2800000,3,3.0,3341.0,Brooklyn,Residential,condo,Brooklyn,11211,New York,40.709601,-73.968217,838.072433
534,2799000,3,3.0,2350.0,Brooklyn,Residential,condo,Brooklyn,11215,New York,40.667604,-73.993127,1191.063830
535,2798000,5,3.0,3400.0,Brooklyn,Residential,multi-family home,Brooklyn,11218,New York,40.657854,-73.975525,822.941176
548,2750000,3,3.0,2782.0,Brooklyn,Commercial,condo,Brooklyn,11231,New York,40.681105,-74.009814,988.497484


In [194]:
df_copy = merged_df.copy()

# Convert column names to lowercase in the original DataFrame
df_copy.columns = df_copy.columns.str.lower()

# Save the merged DataFrame to a CSV file
df_copy.to_csv('resources/Cleaned_data_without_outliers.csv', index=False)

# Confirm the file has been saved
print("File saved as 'Cleaned_data_without_outliers'")


File saved as 'Cleaned_data_without_outliers'
